```
% head -n 10001 Liquor_Sales.csv | cat >Liquor_Sales_10K.csv
```

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import pandas_categorical as pdc
from sales_util import read_liquor_csv, infer_relation_cardinalities

In [2]:
df = read_liquor_csv("Liquor_Sales_10K")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   Invoice/Item Number    10000 non-null  string  
 1   Date                   10000 non-null  category
 2   Store Number           10000 non-null  category
 3   Store Name             10000 non-null  category
 4   Address                10000 non-null  category
 5   City                   10000 non-null  category
 6   Zip Code               10000 non-null  category
 7   Store Location         10000 non-null  category
 8   County Number          9969 non-null   category
 9   County                 10000 non-null  category
 10  Category               9996 non-null   category
 11  Category Name          10000 non-null  category
 12  Vendor Number          10000 non-null  category
 13  Vendor Name            10000 non-null  category
 14  Item Number            10000 non-null  

In [38]:
df.iloc[0]

Invoice/Item Number                              S24127700024
Date                                      2015-02-19 00:00:00
Store Number                                             3678
Store Name                                    Smoke Shop, The
Address                                       1918 SE 14TH ST
City                                               DES MOINES
Zip Code                                                50320
Store Location                   POINT (-93.597011 41.570844)
County Number                                              77
County                                                   Polk
Category                                              1031200
Category Name                                  VODKA FLAVORED
Vendor Number                                             380
Vendor Name                         Phillips Beverage Company
Item Number                                             41783
Item Description                 Uv Blue Raspberry Vodka Mini
Pack    

`Cities` and `Zip Code` relate many-to-many

In [4]:
pd.concat([
    df[df['Zip Code'] == '50322'][['Zip Code', 'City']],
    df[df['City'] == 'URBANDALE'][['Zip Code', 'City']]])

,Zip Code,City
165,50322,URBANDALE
223,50322,DES MOINES
340,50322,URBANDALE
412,50322,DES MOINES
423,50322,URBANDALE
...,...,...
9503,50322,URBANDALE
9705,50322,URBANDALE
9851,50322,URBANDALE
9912,50322,URBANDALE


Explore cardinality of relationships between categorial variables:

In [46]:
infer_relation_cardinalities(df)

,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,Category,Category Name,Vendor Number,Vendor Name,Item Number,Item Description,Pack,Bottle Volume (ml),Full Address,Store,Street & Zip
Date,0,28,28,28,23,25,24,18,18,19,19,19,19,35,33,7,8,28,28,28
Store Number,87,0,2,2,2,2,2,1,2,31,36,26,30,85,83,7,11,2,2,2
Store Name,87,3,0,3,2,2,2,2,2,31,36,26,30,85,83,7,11,3,3,3
Address,113,2,2,0,2,2,2,2,2,33,33,26,26,112,104,7,10,2,2,2
City,490,87,87,84,0,13,71,2,2,54,54,43,43,543,398,10,13,86,87,84
Zip Code,185,31,30,29,4,0,28,2,3,44,50,32,39,209,177,8,12,29,31,29
Store Location,482,94,95,105,67,66,0,41,49,59,74,40,58,530,402,8,12,105,95,105
County Number,695,196,194,202,17,26,160,0,2,60,62,51,53,816,607,11,14,204,196,202
County,694,196,194,201,17,26,160,1,0,60,60,51,51,815,606,11,14,203,196,201
Category,640,655,651,654,222,248,594,96,97,0,2,23,23,138,116,7,9,656,655,654


Many-to_One relationships:
- `Store Number` to `County Number`
- `County` to `County Number`
- `Vendor Name` to `Vendor Number`
- `Item Number` to `Pack`

All other pairs are "many-to-many".

Exploring suspiciously low cardinality between store variables:

In [17]:
df.groupby('Store Name')['Store Number'].nunique()[lambda x: x > 1]

Store Name
B and B West                            2
Bani's                                  2
Broadway Liquor                         2
Emmetsburg Food Pride                   2
Famous Liquors                          2
Gasland #102 / Burlington               2
Huber's Store Inc.                      2
Ida Grove Food Pride                    2
Jeff's Market / Blue Grass              2
Jeff's Market / Durant                  2
Jeff's Market / Wilton                  3
Kellogg Country Store                   2
Kum & Go #443 / Williamsburg            2
Kum & Go #97 / West Des Moines          2
Lake Liquors Wine and Spirits           2
Laurens Food Pride                      2
Liquor And Tobacco Outlet               2
Liquor and Tobacco Outlet /             2
No Frills Supermarkets / Council Blu    2
Osage Liquors                           2
Point Liquor & Tobacco                  2
Sac City Food Pride                     2
Sam's Mini Mart / Sioux City            2
Sauce                  

In [25]:
df[df['Address'] == "101 MAIN ST"].head(3)

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,...,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons),Full Address
632,S16978900009,2014-01-22,4762,Huber's Store Inc.,101 MAIN ST,FORT ATKINSON,52144,POINT (-91.932448 43.14339),96,Winneshiek,...,Disaronno Amaretto,12,750,13.86,20.79,2,41.58,1.5,0.40,101 MAIN ST | FORT ATKINSON | 52144
2029,S14909200001,2013-10-02,4762,Huber's Store Inc.,101 MAIN ST,FORT ATKINSON,52144,POINT (-91.932448 43.14339),96,Winneshiek,...,Dr. Mcgillicuddy's Cherry Schnapps,6,1750,14.66,21.99,6,131.94,10.5,2.77,101 MAIN ST | FORT ATKINSON | 52144
2169,S27278900013,2015-08-12,4862,Pronto Market / Kingsley,101 MAIN ST,KINGSLEY,51028,POINT (-95.967322 42.58707400000001),75,Plymouth,...,Chi-Chi's Strawberry Margarita W/tequila,6,1750,6.50,9.75,6,58.50,10.5,2.77,101 MAIN ST | KINGSLEY | 51028


In [ ]:
df.groupby('Item Description')['Item Number'].nunique()[lambda x: x > 1]

Conclusions:
- A customer (`Store Number`) can have multiple stores (`Store Name`)
- `Address`es are not unique because same street & number exists in different `City`s
- `Store Name`s are reused by differnt customers (`Store Number`). E.g. franchises

In [49]:
df["Full Address"] = df[['Address', 'City', 'Zip Code']].agg(' | '.join, axis=1)
df["Store"] = df['Store Number'].apply(str)
df["Store"] = df[['Store', 'Store Name']].agg(' | '.join, axis=1)
df["Street & Zip"] = df[['Address', 'Zip Code']].agg(' | '.join, axis=1)


None of the above has low cardinality:

In [50]:
df.apply(pd.Series.nunique)

Invoice/Item Number      10000
Date                       840
Store Number              1252
Store Name                1226
Address                   1289
City                       382
Zip Code                   363
Store Location            1084
County Number               99
County                     113
Category                    73
Category Name               87
Vendor Number               70
Vendor Name                 89
Item Number               1574
Item Description          1175
Pack                        12
Bottle Volume (ml)          18
State Bottle Cost         1008
State Bottle Retail       1195
Bottles Sold                44
Sale (Dollars)            3098
Volume Sold (Liters)        97
Volume Sold (Gallons)       97
Full Address              1301
Store                     1254
Street & Zip              1296
dtype: int64

In [57]:
import sqlite3
con = sqlite3.connect("liquor.db")
cur = con.cursor()
#cur.execute("CREATE TABLE county(number INTEGER PRIMARY KEY, name TEXT)")

In [58]:
#cur.execute("CREATE TABLE address(number INTEGER PRIMARY KEY, name TEXT)")

In [59]:
# https://docs.python.org/3/library/hashlib.html
import hashlib
# hashlib.algorithms_available
# https://en.wikipedia.org/wiki/List_of_hash_functions

# md5 is available on every platform and is 128-bit short

STORE_FIELDS = [
    'Store Number',
    'Store Name',
    'Address',                   
    'City',                       
    'Zip Code',                   
    'Store Location',                      
    'County']  


#df["Store"] = df[['Address', 'City', 'Zip Code']].agg('|'.join, axis=1).apply(hashlib.md5)

          
